<a href="https://colab.research.google.com/github/OksanaGlazkova/DI/blob/main/Junior_DE_%D0%93%D0%BB%D0%B0%D0%B7%D0%BA%D0%BE%D0%B2%D0%B0_%D0%9E%D0%BA%D1%81%D0%B0%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

**Добавим немного данных:**

In [ ]:
data = {'customer_id':[1, 1, 1, 1, 2, 2, 2, 2, 2], 'product_id':[10, 11, 12, 13, 14, 15, 16, 17, 18], 'timestamp':['2008-11-18 17:20:19', '2008-11-18 17:21:20', '2008-11-18 17:30:30', '2008-11-18 17:21:21', '2008-11-18 17:25:25', '2008-11-18 17:31:31', '2008-11-18 17:31:31', '2008-11-18 17:38:38', '2008-11-18 17:39:39']}

In [ ]:
df = pd.DataFrame(data)
df

,customer_id,product_id,timestamp
0,1,10,2008-11-18 17:20:19
1,1,11,2008-11-18 17:21:20
2,1,12,2008-11-18 17:30:30
3,1,13,2008-11-18 17:21:21
4,2,14,2008-11-18 17:25:25
5,2,15,2008-11-18 17:31:31
6,2,16,2008-11-18 17:31:31
7,2,17,2008-11-18 17:38:38
8,2,18,2008-11-18 17:39:39


**Добавим уникальные идентификаторы для каждой строки, для дальнейших манипуляций с данными:**

In [ ]:
df['id'] = list(range(len(df.sort_values(['customer_id', 'timestamp']))))
df['id'] = df['id']+1
df

,customer_id,product_id,timestamp,id
0,1,10,2008-11-18 17:20:19,1
1,1,11,2008-11-18 17:21:20,2
2,1,12,2008-11-18 17:30:30,3
3,1,13,2008-11-18 17:21:21,4
4,2,14,2008-11-18 17:25:25,5
5,2,15,2008-11-18 17:31:31,6
6,2,16,2008-11-18 17:31:31,7
7,2,17,2008-11-18 17:38:38,8
8,2,18,2008-11-18 17:39:39,9


In [ ]:
df.dtypes

customer_id     int64
product_id      int64
timestamp      object
id              int64
dtype: object

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.dtypes

customer_id             int64
product_id              int64
timestamp      datetime64[ns]
id                      int64
dtype: object

In [ ]:
session = df.sort_values(['customer_id', 'timestamp'])[['id', 'customer_id','timestamp']]
session

,id,customer_id,timestamp
0,1,1,2008-11-18 17:20:19
1,2,1,2008-11-18 17:21:20
3,4,1,2008-11-18 17:21:21
2,3,1,2008-11-18 17:30:30
4,5,2,2008-11-18 17:25:25
5,6,2,2008-11-18 17:31:31
6,7,2,2008-11-18 17:31:31
7,8,2,2008-11-18 17:38:38
8,9,2,2008-11-18 17:39:39


In [ ]:
session['diffs'] = session.groupby('customer_id')['timestamp'].diff().dt.seconds
session

,id,customer_id,timestamp,diffs
0,1,1,2008-11-18 17:20:19,NaN
1,2,1,2008-11-18 17:21:20,61.0
3,4,1,2008-11-18 17:21:21,1.0
2,3,1,2008-11-18 17:30:30,549.0
4,5,2,2008-11-18 17:25:25,NaN
5,6,2,2008-11-18 17:31:31,366.0
6,7,2,2008-11-18 17:31:31,0.0
7,8,2,2008-11-18 17:38:38,427.0
8,9,2,2008-11-18 17:39:39,61.0


In [ ]:
sessions_start = session[(session['diffs'].isnull()) | (session['diffs'] > 180)]
sessions_start['session_id'] = sessions_start['id']
sessions_start

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,customer_id,timestamp,diffs,session_id
0,1,1,2008-11-18 17:20:19,NaN,1
2,3,1,2008-11-18 17:30:30,549.0,3
4,5,2,2008-11-18 17:25:25,NaN,5
5,6,2,2008-11-18 17:31:31,366.0,6
7,8,2,2008-11-18 17:38:38,427.0,8


In [ ]:
df = df.sort_values('id')
sessions_start = sessions_start.sort_values('id')
df = pd.merge_asof(df, sessions_start[['id','customer_id', 'session_id']], on='id' ,by='customer_id')
df

,customer_id,product_id,timestamp,id,session_id
0,1,10,2008-11-18 17:20:19,1,1
1,1,11,2008-11-18 17:21:20,2,1
2,1,12,2008-11-18 17:30:30,3,3
3,1,13,2008-11-18 17:21:21,4,3
4,2,14,2008-11-18 17:25:25,5,5
5,2,15,2008-11-18 17:31:31,6,6
6,2,16,2008-11-18 17:31:31,7,6
7,2,17,2008-11-18 17:38:38,8,8
8,2,18,2008-11-18 17:39:39,9,8


**Приводим к изначальному виду уже с session_id:**

In [ ]:
df = df[['customer_id', 'product_id', 'timestamp', 'session_id']]
df

,customer_id,product_id,timestamp,session_id
0,1,10,2008-11-18 17:20:19,1
1,1,11,2008-11-18 17:21:20,1
2,1,12,2008-11-18 17:30:30,3
3,1,13,2008-11-18 17:21:21,3
4,2,14,2008-11-18 17:25:25,5
5,2,15,2008-11-18 17:31:31,6
6,2,16,2008-11-18 17:31:31,6
7,2,17,2008-11-18 17:38:38,8
8,2,18,2008-11-18 17:39:39,8


**session_id в формате int**

In [ ]:
df.dtypes

customer_id             int64
product_id              int64
timestamp      datetime64[ns]
session_id              int64
dtype: object